In [12]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

from webdriver_manager.chrome import ChromeDriverManager

from time import sleep

import os
import gzip
import shutil

In [13]:
uname=os.getlogin()

carp1 = 'genoma'
newpath1 = 'C:\\Users\\{}\\Downloads\\{}'.format(uname,carp1)
if not os.path.exists(newpath1):
    os.makedirs(newpath1)

carp2 = 'proteina'
newpath2 = 'C:\\Users\\{}\\Downloads\\{}'.format(uname,carp2)
if not os.path.exists(newpath2):
    os.makedirs(newpath2)

opt=Options()

opt.add_argument("--headless")
prefs = {
    "profile.default_content_settings.popups": 0,
    "download.prompt_for_download": False,
    "download.directory_upgrade ": True,
    'download.default_directory': 'C:\\Users\\{}\\Downloads'.format(uname),
}
opt.add_experimental_option('prefs', prefs)


driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=opt
)
url=input  ('Seleccionana en la página del ncib (https://www.ncbi.nlm.nih.gov/datasets/genome/) \n\
            los taxones y filtros que requieras, copia y pega completa la url resultante aquí:')
driver.get(url)

sleep(5)

try:
    al = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section[3]/div[2]/div[1]/section[3]/div/div[3]/div').get_attribute("aria-label")
except Exception:
    WebDriverWait(driver, 60)\
        .until(EC.element_to_be_clickable((By.XPATH,"//*[text()='No Thanks']")))\
        .click()
    al = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section[3]/div[2]/div[1]/section[3]/div/div[3]/div').get_attribute("aria-label")

h=int(al.replace('Showing 1 to 20 of ','').replace(',',''))
m= h//20
r= h%30
if r!=0:
    p=m+1
else:
    p=m
d=p-1

j=0
n=20

for k in range(p):
    for i in range(n):
        try:
            WebDriverWait(driver, 60)\
                .until(EC.element_to_be_clickable((By.XPATH,'/html/body/div[3]/main/div/section[3]/div[2]/div[2]/div/div/div[2]/div/table/tbody/tr[{}]/td[12]/button'.format(str(i+1)))))\
                .click()
        except Exception:
            WebDriverWait(driver, 60)\
                .until(EC.element_to_be_clickable((By.XPATH,"//*[text()='No Thanks']")))\
                .click()
            WebDriverWait(driver, 60)\
                .until(EC.element_to_be_clickable((By.XPATH,'/html/body/div[3]/main/div/section[3]/div[2]/div[2]/div/div/div[2]/div/table/tbody/tr[{}]/td[12]/button'.format(str(i+1)))))\
                .click()

        try:
            WebDriverWait(driver, 60)\
                .until(EC.element_to_be_clickable((By.XPATH, "//*[text()='View details']")))\
                .click()
        except Exception:
            WebDriverWait(driver, 60)\
                .until(EC.element_to_be_clickable((By.XPATH,"//*[text()='No Thanks']")))\
                .click()
            WebDriverWait(driver, 60)\
                .until(EC.element_to_be_clickable((By.XPATH, "//*[text()='View details']")))\
                .click()

        WebDriverWait(driver, 60).until(EC.number_of_windows_to_be(2))
        driver.switch_to.window(driver.window_handles[-1])

        try:
            WebDriverWait(driver, 60)\
                .until(EC.element_to_be_clickable((By.XPATH,'//*[@id="show-link-menu-button"]')))\
                .click()
        except Exception:
            WebDriverWait(driver, 60)\
                .until(EC.element_to_be_clickable((By.XPATH,"//*[text()='No Thanks']")))\
                .click()
            WebDriverWait(driver, 60)\
                .until(EC.element_to_be_clickable((By.XPATH,'//*[@id="show-link-menu-button"]')))\
                .click()

        try:
            WebDriverWait(driver, 60)\
                .until(EC.element_to_be_clickable((By.LINK_TEXT, "See more files on FTP")))\
                .click()
        except Exception:
            WebDriverWait(driver, 60)\
                .until(EC.element_to_be_clickable((By.XPATH,"//*[text()='No Thanks']")))\
                .click()
            WebDriverWait(driver, 60)\
                .until(EC.element_to_be_clickable((By.LINK_TEXT, "See more files on FTP")))\
                .click()

        try:

            sleep(5)
            st=0

            file_1 = driver.find_element(By.PARTIAL_LINK_TEXT, "_cds_from_genomic.fna.gz")
            WebDriverWait(driver, 60)\
                .until(EC.element_to_be_clickable((By.XPATH,'//*[@href="{}"]'.format(file_1.text))))\
                .click()

            st+=1

            file_2 = driver.find_element(By.PARTIAL_LINK_TEXT, "_protein.faa.gz")
            WebDriverWait(driver, 60)\
                .until(EC.element_to_be_clickable((By.XPATH,'//*[@href="{}"]'.format(file_2.text))))\
                .click()

            st+=1

        except Exception:
            j+=1
            print('Falta un archivo. No se descarga en'+ driver.title)
            print('Van', j,'cepas que no tienen los archivos deseados')

        if st!=0:
            sleep(5)
            with gzip.open("C:\\Users\\{}\\Downloads\\{}".format(uname, file_1.text), 'rt') as f_in:
                with open("C:\\Users\\{}\\Downloads\\{}\\{}".format(uname, carp1, file_1.text.replace('.gz','')), 'wt') as f_out:
                    shutil.copyfileobj(f_in, f_out)

            if st==2:
                with gzip.open("C:\\Users\\{}\\Downloads\\{}".format(uname, file_2.text), 'rt') as f_in:
                    with open("C:\\Users\\{}\\Downloads\\{}\\{}".format(uname, carp2, file_2.text.replace('.gz','')), 'wt') as f_out:
                        shutil.copyfileobj(f_in, f_out)

        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        driver.switch_to.active_element.send_keys(Keys.ESCAPE)
    if k!=d:
        try:
            WebDriverWait(driver, 60)\
                .until(EC.element_to_be_clickable((By.XPATH,'/html/body/div[3]/main/div/section[3]/div[2]/div[1]/section[3]/div/div[4]/button[2]')))\
                .click()
        except Exception:
            WebDriverWait(driver, 60)\
                .until(EC.element_to_be_clickable((By.XPATH,"//*[text()='No Thanks']")))\
                .click()
            WebDriverWait(driver, 60)\
                .until(EC.element_to_be_clickable((By.XPATH,'/html/body/div[3]/main/div/section[3]/div[2]/div[1]/section[3]/div/div[4]/button[2]')))\
                .click()
    sleep(5)
driver.quit()

Seleccionana en la página del ncib (https://www.ncbi.nlm.nih.gov/datasets/genome/) 
            los taxones y filtros que requieras, copia y pega completa la url resultante aquí: https://www.ncbi.nlm.nih.gov/datasets/genome/?taxon=77643&assembly_level=3%3A3&release_year=2013%3A2023


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00974DE3+43907]
	(No symbol) [0x00900741]
	(No symbol) [0x007F33ED]
	(No symbol) [0x00826760]
	(No symbol) [0x00826C2B]
	(No symbol) [0x00856F62]
	(No symbol) [0x00842BA4]
	(No symbol) [0x008555CA]
	(No symbol) [0x00842956]
	(No symbol) [0x0081E17E]
	(No symbol) [0x0081F32D]
	GetHandleVerifier [0x00C25AF9+2865305]
	GetHandleVerifier [0x00C6E78B+3163435]
	GetHandleVerifier [0x00C68441+3138017]
	GetHandleVerifier [0x009FE0F0+605840]
	(No symbol) [0x0090A64C]
	(No symbol) [0x00906638]
	(No symbol) [0x0090675F]
	(No symbol) [0x008F8DB7]
	BaseThreadInitThunk [0x75C47BA9+25]
	RtlInitializeExceptionChain [0x7708BD2B+107]
	RtlClearBits [0x7708BCAF+191]
